In [ ]:
## Project 3: Scraping expensive flats

Veera Tegelberg
veera.tegelberg@gmail.com

In this project, I scraped a Finnish home listing web page called Oikotie by using Playwright.

In the end of July 2024, there were 22 tabs with 514 listings (24 apartments per page).

My goal was to collect information of all listed apartments worth at least €1,000,000, clean the data by using tidy data principles and investigate following questions:
- Where are the expensice apartments located (municipality, address)?
- What is the most expensive flat?
- Smallest vs. biggest?
- An avarage size?

In addition to scraping, I wanted to practice data frame building, cleaning, analyzing and storing data by using Python.

In [1]:
# Import needed libraries.

import pandas as pd
from playwright.async_api import async_playwright, expect
import asyncio

In [2]:
# THIS CODE INCLUDES CALLING THE PLAYWRIGHT AND FORMATING THE DF.

# Using asynchronous version
async def open_browser(headless=False):

    
    playwright = await async_playwright().start()

    # Scraping with Chrome.
    browser = await playwright.chromium.launch(headless=headless)

    page = await browser.new_page()

    return browser, page

driver, page = await open_browser()

# I have manually chosen only apartments located in Finland (Suomi) and cost at least €1,000,000.
url = "https://asunnot.oikotie.fi/myytavat-asunnot?pagination={}&locations=%5B%5B1,9,%22Suomi%22%5D%5D&price%5Bmin%5D=1000000&cardType=100"

# Giving column names.
df = pd.DataFrame(columns=['address', 'price', 'size'])

# First for loop to go through pages from 1 to 22 (I have manually checked the amount)
# In url there is a placeholder {}
for i in range(1, 23):  
    current_url = url.format(i)
    await page.goto(current_url)

    # Adding a one-second delay after navigating to the page just to be polite
    await asyncio.sleep(1)

    # Using 'try - except' to go through the page, even some info would be missing.
    try:
        # Waiting for the price and size elements to load for 15 seconds just in case
        # I have found the attributes with xpath
        await page.wait_for_selector('.card-v2-text-container__title', timeout=15000)
        await page.wait_for_selector('.card-v2-text-container__text--bold', timeout=15000)

        # Extracting all text and address elements on the page
        text_elements = await page.query_selector_all('.card-v2-text-container__title')
        address_elements = await page.query_selector_all('.card-v2-text-container__text--bold')

        listings = []
        # Second for loop to go through elements
        for index, text_element in enumerate(text_elements):
            text = await text_element.inner_text()
            # In listings there are prices (€) and monthly dues (€/kk (€/month)). I exclude monthly dues from results.
            if text.endswith('€') and not text.endswith('€/kk'):
                price = text

                # price and size seemed to have same xpath, so I have to get size like this
                for next_element in text_elements[index + 1:]:
                    size_text = await next_element.inner_text()
                    # Using square meter unit to pick up size information
                    if 'm²' in size_text:
                        size = size_text
                        break
                address = await address_elements.pop(0).inner_text() if address_elements else 'N/A'
                listings.append({'price': price, 'size': size, 'address': address})

        # Converting the list of listings to a df and concatenate to the main df
        df = pd.concat([df, pd.DataFrame(listings)], ignore_index=True)
        # Getting feedback while scraping to see, if information is found
        print(f"Page {i} - Found {len(listings)} listings")
    except Exception as e:
        print(f"Page {i} - Error: {e}")

df

Page 1 - Found 24 listings
Page 2 - Found 24 listings
Page 3 - Found 24 listings
Page 4 - Found 24 listings
Page 5 - Found 24 listings
Page 6 - Found 24 listings
Page 7 - Found 24 listings
Page 8 - Found 24 listings
Page 9 - Found 24 listings
Page 10 - Found 24 listings
Page 11 - Found 24 listings
Page 12 - Found 24 listings
Page 13 - Found 24 listings
Page 14 - Found 24 listings
Page 15 - Found 24 listings
Page 16 - Found 24 listings
Page 17 - Found 24 listings
Page 18 - Found 24 listings
Page 19 - Found 24 listings
Page 20 - Found 24 listings
Page 21 - Found 24 listings
Page 22 - Found 6 listings


,address,price,size
0,"Korkeavuorenkatu 27 A, Kaartinkaupunki, Helsinki",3 280 000 €,173 m²
1,"Lönnrotinkatu 22 A, Kamppi, Helsinki",1 275 000 €,144 m²
2,"Fredrikinkatu 16 A, Punavuori, Helsinki",1 490 000 €,156 m²
3,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Punavu...",1 123 200 €,108 m²
4,"Hietalahdenranta 2 A 69, Punavuori, Helsinki",2 941 500 €,159 m²
...,...,...,...
505,"Kalevan Puistotie 24 A 53, Tammela, Tampere",1 378 500 €,150 m²
506,"Rypysuontie 6, Rypysuo, Kuopio",1 150 000 €,235 m²
507,"Kansikatu 1 B116, Keskusta, Tampere",2 369 230 €,"192,5 m²"
508,"Kesäkatu 2, Taka-Töölö, Helsinki",2 400 000 €,"169,5 m²"


In [3]:
# Saving result on my computer
df.to_csv('listings240804.csv', index=False)

#2 Data cleaning

- From 'address', make a new columns called 'borough', 'city' and 'street_address'
- Delete original 'address' and make new one by combining 'street_address' and 'city'.
- From 'price' drop away '€' and clean spaces
- From 'size', drop away the " m²" and change commas to dots.
- Make a new column called "€/m²". --> Divide price with size.

In [4]:
# Just in case, making a new df of the saved stuff, so I will not mess it up accidentially.
df1 = pd.read_csv("listings240804.csv")
df1

,address,price,size
0,"Korkeavuorenkatu 27 A, Kaartinkaupunki, Helsinki",3 280 000 €,173 m²
1,"Lönnrotinkatu 22 A, Kamppi, Helsinki",1 275 000 €,144 m²
2,"Fredrikinkatu 16 A, Punavuori, Helsinki",1 490 000 €,156 m²
3,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Punavu...",1 123 200 €,108 m²
4,"Hietalahdenranta 2 A 69, Punavuori, Helsinki",2 941 500 €,159 m²
...,...,...,...
505,"Kalevan Puistotie 24 A 53, Tammela, Tampere",1 378 500 €,150 m²
506,"Rypysuontie 6, Rypysuo, Kuopio",1 150 000 €,235 m²
507,"Kansikatu 1 B116, Keskusta, Tampere",2 369 230 €,"192,5 m²"
508,"Kesäkatu 2, Taka-Töölö, Helsinki",2 400 000 €,"169,5 m²"


In [5]:
# Split the 'address' into multiple columns
split_columns = df1['address'].str.split(', ', expand=True)

# Create new 'borough' and 'city' columns based on the split columns
df1['street_address'] = split_columns[0]
df1['borough'] = split_columns[1]
df1['city'] = split_columns[2]

# On some rows, 'city' is NaN, because 'borough' is not mentioned in the listing on the web page. 
# If 'city' == NaN, copy 'borough' to 'city'.
df1['city'] = df1['city'].fillna(df1['borough'])

# Checking, that there are no 'NaN' in 'city' anymore.
nan_count = df1['city'].isnull().sum()
print(f"Number of NaN values in 'city' column: {nan_count}")

# Display the DataFrame to verify the result
df1.head()

Number of NaN values in 'city' column: 0


,address,price,size,street_address,borough,city
0,"Korkeavuorenkatu 27 A, Kaartinkaupunki, Helsinki",3 280 000 €,173 m²,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki
1,"Lönnrotinkatu 22 A, Kamppi, Helsinki",1 275 000 €,144 m²,Lönnrotinkatu 22 A,Kamppi,Helsinki
2,"Fredrikinkatu 16 A, Punavuori, Helsinki",1 490 000 €,156 m²,Fredrikinkatu 16 A,Punavuori,Helsinki
3,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Punavu...",1 123 200 €,108 m²,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki
4,"Hietalahdenranta 2 A 69, Punavuori, Helsinki",2 941 500 €,159 m²,Hietalahdenranta 2 A 69,Punavuori,Helsinki


In [7]:
# Making a new df.
df2 = df1

In [9]:
# Deleting original 'address' 
# Combining 'street_address' and 'city' into new 'address'
# Making a new df 'df3'

df3 = df2.drop(['address'], axis=1).assign(address=df2['street_address'] + ', ' + df2['city'])
df3.head()

,price,size,street_address,borough,city,address
0,3 280 000 €,173 m²,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki"
1,1 275 000 €,144 m²,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki"
2,1 490 000 €,156 m²,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki"
3,1 123 200 €,108 m²,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki"
4,2 941 500 €,159 m²,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki"


In [10]:
# Just checking the datatypes. 'price' and 'size' must be changed into integer and float.
df3.dtypes

price             object
size              object
street_address    object
borough           object
city              object
address           object
dtype: object

In [11]:
# Remove '€' from 'price' and replace spaces with comma.
# Remove 'm²' from 'size and replace comma with dot.

df3['price'] = df3['price'].astype(str)

# I have to use regex to replace spaces, because the web page did not use a 'normal' space.
df4 = df3.assign(
    price = df3['price'].str.replace(r'\s+€', '', regex=True).str.replace(r'\s+', '', regex=True),
    size = df3['size'].str.replace(r'\s+m²', '', regex=True).str.replace(r',', '.', regex=True).str.replace(r' ', '', regex=True)
)

df4.head()

,price,size,street_address,borough,city,address
0,3280000,173,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki"
1,1275000,144,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki"
2,1490000,156,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki"
3,1123200,108,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki"
4,2941500,159,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki"


In [12]:
# I notice, that there are also some building grounds for sale, and they give me an error
# I drop away rows including Finnish word for a building lot ('tontti')
df4 = df4[~df4['size'].str.contains('tontti', case=False)]

# I also know, that there is at least one value that starts '12345...'
# It is not a real apartment, but indicades buyers to offer some kind of amount for the seller.
df4 = df4[~df4['price'].str.startswith('12345')]

df4
# Number of rows decreases.

,price,size,street_address,borough,city,address
0,3280000,173,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki"
1,1275000,144,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki"
2,1490000,156,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki"
3,1123200,108,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki"
4,2941500,159,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki"
...,...,...,...,...,...,...
505,1378500,150,Kalevan Puistotie 24 A 53,Tammela,Tampere,"Kalevan Puistotie 24 A 53, Tampere"
506,1150000,235,Rypysuontie 6,Rypysuo,Kuopio,"Rypysuontie 6, Kuopio"
507,2369230,192.5,Kansikatu 1 B116,Keskusta,Tampere,"Kansikatu 1 B116, Tampere"
508,2400000,169.5,Kesäkatu 2,Taka-Töölö,Helsinki,"Kesäkatu 2, Helsinki"


In [15]:
# change price into int and size to float
df4['price'] = df4['price'].astype(int)
df4['size'] = df4['size'].astype(float)
df4.head()

,price,size,street_address,borough,city,address
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki"
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki"
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki"
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki"
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki"


In [14]:
df5 = df4
df5.head()

,price,size,street_address,borough,city,address
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki"
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki"
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki"
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki"
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki"


In [17]:
# Make a new columns called '€/m²'.

df5['e/m²'] = round(df5['price'] / df5['size'])
df5.head()


,price,size,street_address,borough,city,address,e/m²
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki",18960.0
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki",8854.0
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki",9551.0
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki",10400.0
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki",18500.0


In [18]:
# Checking that the datatypes are ok.
df5.dtypes

price               int32
size              float64
street_address     object
borough            object
city               object
address            object
e/m²              float64
dtype: object

In [19]:
# Saving a new, cleaned version on my computer as csv

df5.to_csv('listings_cleaned240804.csv', index=False)

Before analysis, I did some crosschecking between original and cleaned csvs, so that 'tontti' is really dropped away and that the data looks good.

In [ ]:
#3 Analyzing the data

- Most expensive and cheapest apartment (price and €/m2)? Average and median price?
- Biggest and smallest apartments?
- In which city there are most listing?
- On which street there are most listings?


In [20]:
df6 = pd.read_csv('listings_cleaned240804.csv')
df6.head()

,price,size,street_address,borough,city,address,e/m²
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki",18960.0
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki",8854.0
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki",9551.0
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki",10400.0
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki",18500.0


In [21]:
# Most expensive and cheapest apartment (price)

most_expensive_price = df6.loc[df6['price'].idxmax()]
print(f"The most expensive flat's info looks like this: {most_expensive_price}")

print("- - - - - - -")

cheapest_price = df6.loc[df6['price'].idxmin()]
print(f"The most cheapest apartment's info looks like this: {cheapest_price}")

The most expensive flat's info looks like this: price                                    10950000
size                                        367.0
street_address              Korkeavuorenkatu 21 A
borough                                Ullanlinna
city                                     Helsinki
address           Korkeavuorenkatu 21 A, Helsinki
e/m²                                      29837.0
Name: 309, dtype: object
- - - - - - -
The most cheapest apartment's info looks like this: price                                  1003500
size                                     111.5
street_address           Itätuulenkuja 10 C153
borough                                Tapiola
city                                     Espoo
address           Itätuulenkuja 10 C153, Espoo
e/m²                                    9000.0
Name: 179, dtype: object


In [22]:
# Print top 10 highest prices per square meter.
df6.sort_values('e/m²', ascending=False).head(10)

,price,size,street_address,borough,city,address,e/m²
309,10950000,367.0,Korkeavuorenkatu 21 A,Ullanlinna,Helsinki,"Korkeavuorenkatu 21 A, Helsinki",29837.0
271,2200000,74.0,Vähä Meilahti,Vähä Meilahti,Helsinki,"Vähä Meilahti, Helsinki",29730.0
444,1190000,48.5,Kylväjänkuja 5,Hämeenkylä,Vantaa,"Kylväjänkuja 5, Vantaa",24536.0
293,4350000,194.0,Ehrenströmintie 10 A,Kaivopuisto,Helsinki,"Ehrenströmintie 10 A, Helsinki",22423.0
121,7150000,322.5,Eteläranta 4 A,Kaartinkaupunki,Helsinki,"Eteläranta 4 A, Helsinki",22171.0
283,5346000,243.0,Eteläranta 4 A,Kaartinkaupunki,Helsinki,"Eteläranta 4 A, Helsinki",22000.0
395,3050000,142.5,Eteläranta 4 A,Kaartinkaupunki,Helsinki,"Eteläranta 4 A, Helsinki",21404.0
122,5495000,260.0,Eteläranta 4 A,Kaartinkaupunki,Helsinki,"Eteläranta 4 A, Helsinki",21135.0
64,1955000,97.0,Unioninkatu 5 B,Kaartinkaupunki,Helsinki,"Unioninkatu 5 B, Helsinki",20155.0
493,1650000,83.5,Alvar Aallon katu 5 A,Kluuvi,Helsinki,"Alvar Aallon katu 5 A, Helsinki",19760.0


This top 10 displayed above would have been a good base for a story too.

In [23]:
# Print top 10 lowest prices per square meter.
df6.sort_values('e/m²', ascending=True).head(10)

,price,size,street_address,borough,city,address,e/m²
235,1100000,2361.0,Honkatie 9,Honkalampi,Liperi,"Honkatie 9, Liperi",466.0
383,1900000,3600.0,Kurimontie 39 A,Siikaranta kurimo,Suomussalmi,"Kurimontie 39 A, Suomussalmi",528.0
468,1780000,2862.0,Lehtimäentie 2-16,Kaukas,Hyvinkää,"Lehtimäentie 2-16, Hyvinkää",622.0
324,1780000,2862.0,Lehtimäentie 2-18,Kaukas,Hyvinkää,"Lehtimäentie 2-18, Hyvinkää",622.0
363,1550000,2156.5,Selkostie 3-4,Nilonkangas,Kuusamo,"Selkostie 3-4, Kuusamo",719.0
492,1280000,1414.0,Antinkärki 8,Antinniemi,Valkeakoski,"Antinkärki 8, Valkeakoski",905.0
190,1100000,1149.0,Karhulan Hovintie 1,Kotka,Kotka,"Karhulan Hovintie 1, Kotka",957.0
314,1198000,1134.0,Mikonkatu 12,Keskusta,Mikkeli,"Mikonkatu 12, Mikkeli",1056.0
373,1280000,858.0,Rantatie 708,Pyhäniemi,Hollola,"Rantatie 708, Hollola",1492.0
360,1450000,826.0,Saunaniemenrinne 2,Störsvik Pickala,Siuntio,"Saunaniemenrinne 2, Siuntio",1755.0


In [24]:
# Average price
mean_price = round(df6['price'].mean())
print(f"The average listing's price is {mean_price} euros")

print("- - - - - - -")

# Median price
median_price = round(df6['price'].median())
print(f"The median listing's price is {median_price} euros")

print("- - - - - - -")

# Average square-meter price
mean_m2 = round(df6['e/m²'].mean())
print(f"The average listing's price is {mean_m2} e/m2")

print("- - - - - - -")

# Median square-meter price
median_m2 = round(df6['e/m²'].median())
print(f"The median listing's price is {median_m2} e/m2")

The average listing's price is 1714376 euros
- - - - - - -
The median listing's price is 1367050 euros
- - - - - - -
The average listing's price is 9612 e/m2
- - - - - - -
The median listing's price is 9276 e/m2


In [25]:
# Biggest and smallest apartments?
smallest = df6['size'].min()
print(f"The smallest listed size is {smallest} m2.")

biggest = df6['size'].max()
print(f"The biggest listed size is {biggest} m2.")

average_size = round(df6['size'].mean())
print(f"The average listed size is {average_size} m2.")

median_size = round(df6['size'].median())
print(f"The median listed size is {median_size} m2.")

The smallest listed size is 48.5 m2.
The biggest listed size is 3600.0 m2.
The average listed size is 229 m2.
The median listed size is 166 m2.


The median size of listed, pricy apartments is 166 m2 (1 776 square feet). This is more than double the size of an average household. According to Statistics Finland, in 2023 the average flat size was 80 m2 (861 square feet). https://pxdata.stat.fi/PxWeb/pxweb/fi/StatFin/StatFin__asas/statfin_asas_pxt_115a.px/table/tableViewLayout1/ 

In [28]:
# Top 10 biggest
df6.sort_values('size', ascending=False).head(10)

,price,size,street_address,borough,city,address,e/m²
383,1900000,3600.0,Kurimontie 39 A,Siikaranta kurimo,Suomussalmi,"Kurimontie 39 A, Suomussalmi",528.0
324,1780000,2862.0,Lehtimäentie 2-18,Kaukas,Hyvinkää,"Lehtimäentie 2-18, Hyvinkää",622.0
468,1780000,2862.0,Lehtimäentie 2-16,Kaukas,Hyvinkää,"Lehtimäentie 2-16, Hyvinkää",622.0
235,1100000,2361.0,Honkatie 9,Honkalampi,Liperi,"Honkatie 9, Liperi",466.0
363,1550000,2156.5,Selkostie 3-4,Nilonkangas,Kuusamo,"Selkostie 3-4, Kuusamo",719.0
228,2538000,1423.0,Rantakatu 18,Keskusta,Vaasa,"Rantakatu 18, Vaasa",1784.0
492,1280000,1414.0,Antinkärki 8,Antinniemi,Valkeakoski,"Antinkärki 8, Valkeakoski",905.0
190,1100000,1149.0,Karhulan Hovintie 1,Kotka,Kotka,"Karhulan Hovintie 1, Kotka",957.0
314,1198000,1134.0,Mikonkatu 12,Keskusta,Mikkeli,"Mikonkatu 12, Mikkeli",1056.0
490,5500000,933.0,Pähkinäkukkula 174,Hollola,Hollola,"Pähkinäkukkula 174, Hollola",5895.0


It is possible to notice, that most big listings are located in small municipalities. Most likely they are entire housing units. --> I investigate listings manually, and so it seems.

In [29]:
# Top 10 smalles listing per 'size'.
df6.sort_values('size', ascending=True).head(10)

,price,size,street_address,borough,city,address,e/m²
444,1190000,48.5,Kylväjänkuja 5,Hämeenkylä,Vantaa,"Kylväjänkuja 5, Vantaa",24536.0
271,2200000,74.0,Vähä Meilahti,Vähä Meilahti,Helsinki,"Vähä Meilahti, Helsinki",29730.0
91,1090000,75.0,Kalliolinnantie 10 B,Kaivopuisto,Helsinki,"Kalliolinnantie 10 B, Helsinki",14533.0
138,1275000,75.5,Bulevardi 14 B,Kamppi,Helsinki,"Bulevardi 14 B, Helsinki",16887.0
342,1469000,78.0,Yrjönkatu 2 A,Punavuori,Helsinki,"Yrjönkatu 2 A, Helsinki",18833.0
327,1066500,79.0,Tehtaankatu 12 E,Ullanlinna,Helsinki,"Tehtaankatu 12 E, Helsinki",13500.0
97,1200000,81.0,Unioninkatu 5 B 11,Kaartinkaupunki,Helsinki,"Unioninkatu 5 B 11, Helsinki",14815.0
102,1142000,82.0,Armfeltintie 7-9,Eira,Helsinki,"Armfeltintie 7-9, Helsinki",13927.0
493,1650000,83.5,Alvar Aallon katu 5 A,Kluuvi,Helsinki,"Alvar Aallon katu 5 A, Helsinki",19760.0
308,1548750,85.5,Unioninkatu 5 B,Kaartinkaupunki,Helsinki,"Unioninkatu 5 B, Helsinki",18114.0


In [30]:
# In which city there are most listing?
# To get both columns after value_counts into a csv, I need to turn results into a df with reset_index().
listings_per_city = df6.city.value_counts().reset_index()
# Naming the columns.
listings_per_city.columns = ['city', 'count']

print(listings_per_city)
print("- - - - -")
print(len(listings_per_city))

# Save result to its own csv.
listings_per_city.to_csv('listings-per-city240804.csv', index=False)


                city  count
0           Helsinki    268
1              Espoo     99
2            Tampere     20
3              Turku     18
4         Kauniainen     12
5        Kirkkonummi      8
6           Naantali      6
7              Hanko      4
8               Oulu      4
9            Siuntio      4
10            Porvoo      4
11         Raasepori      3
12         Jyväskylä      3
13           Hollola      2
14          Hyvinkää      2
15          Parainen      2
16           Kaarina      2
17             Inkoo      2
18            Vantaa      2
19           Mikkeli      2
20             Lohja      2
21         Enontekiö      2
22            Kuopio      2
23             Sipoo      2
24          Ylöjärvi      2
25            Hamina      1
26           Kuusamo      1
27           Luumäki      1
28           Hattula      1
29       Suomussalmi      1
30        Tammisaari      1
31          Lempäälä      1
32       Valkeakoski      1
33  Helsinki-Uusimaa      1
34             Kotka

Helsinki is totally dominating when it comes to the amount of expensive flats! More than half of the flats are in the capital. One main reason must be the land price.

In [31]:
# On which street there are most listings?

# 'street_address' includes street and house and apartment number. 
# I want to get rid of all digits and characters coming after them (e.g. "Kalliolinnantie 10 B" --> "Kalliolinnantie")
# Make a new column called 'street'.

df6 = df5

# Creating a new column 'street' by extracting characters before any digits in 'street_address'
# ^ starts from the beginning of value, [^\d] commands the code to find anything exept a number (d=digit)
df6['street'] = df5['street_address'].str.extract(r'^([^\d]+)')

# Remove any spaces from the extracted street names
df6['street'] = df5['street'].str.strip()

# Display the updated DataFrame
df6

,price,size,street_address,borough,city,address,e/m²,street
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki",18960.0,Korkeavuorenkatu
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki",8854.0,Lönnrotinkatu
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki",9551.0,Fredrikinkatu
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki",10400.0,Punavuorenkatu
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki",18500.0,Hietalahdenranta
...,...,...,...,...,...,...,...,...
505,1378500,150.0,Kalevan Puistotie 24 A 53,Tammela,Tampere,"Kalevan Puistotie 24 A 53, Tampere",9190.0,Kalevan Puistotie
506,1150000,235.0,Rypysuontie 6,Rypysuo,Kuopio,"Rypysuontie 6, Kuopio",4894.0,Rypysuontie
507,2369230,192.5,Kansikatu 1 B116,Keskusta,Tampere,"Kansikatu 1 B116, Tampere",12308.0,Kansikatu
508,2400000,169.5,Kesäkatu 2,Taka-Töölö,Helsinki,"Kesäkatu 2, Helsinki",14159.0,Kesäkatu


In [32]:
# Making a new df to check, how many times each street is recurring in the data

# Create a new df with 'street' and 'city' columns
street_and_city = df6[['street', 'city']].copy()

common_streets = street_and_city.groupby(['street', 'city']).size().reset_index(name='count').sort_values(by='count', ascending=False)

# Combine 'street' and 'city' into a new column, called 'address'.
common_streets['address'] = common_streets['street'] + ', ' + common_streets['city']

print(common_streets[common_streets['count'] >= 5])
print("- - - - - - - -")
print(f"There are {len(common_streets[common_streets['count'] >= 5])} streets, on which there are at least 5 expensive flats for sale.")

# Saving results into csv.
common_streets.to_csv("listings-per-street240804.csv", index=False)

                 street      city  count                       address
109  Konttinosturinkuja  Helsinki     13  Konttinosturinkuja, Helsinki
53           Horisontti  Helsinki     11          Horisontti, Helsinki
260         Tehtaankatu  Helsinki     11         Tehtaankatu, Helsinki
299        Westendintie     Espoo     10           Westendintie, Espoo
48     Hietalahdenranta  Helsinki     10    Hietalahdenranta, Helsinki
162            Merikatu  Helsinki     10            Merikatu, Helsinki
70    Jousenpuistonkatu     Espoo      9      Jousenpuistonkatu, Espoo
67        Itätuulenkuja     Espoo      8          Itätuulenkuja, Espoo
271         Tuuliniitty     Espoo      7            Tuuliniitty, Espoo
35        Fredrikinkatu  Helsinki      7       Fredrikinkatu, Helsinki
273          Töllinmäki  Helsinki      6          Töllinmäki, Helsinki
276         Unioninkatu  Helsinki      6         Unioninkatu, Helsinki
22        Capellanranta  Helsinki      5       Capellanranta, Helsinki
32    

This street data above could have also been base for a story. Most ovious reason for several results per street is, that there are new expensive houses built, but not all apartments are yet sold.

LIMITING THE DATA!!

I dig the data and realize, it should be useful to investigate only listings smaller than 800 m2. It seems that listings bigger than that are not one-family apartments but bigger units.
So, I make a new df with apartments smaller than 800 m2.

In [34]:
df7 = pd.read_csv('listings_cleaned240804.csv')
df7.head()


,price,size,street_address,borough,city,address,e/m²
0,3280000,173.0,Korkeavuorenkatu 27 A,Kaartinkaupunki,Helsinki,"Korkeavuorenkatu 27 A, Helsinki",18960.0
1,1275000,144.0,Lönnrotinkatu 22 A,Kamppi,Helsinki,"Lönnrotinkatu 22 A, Helsinki",8854.0
2,1490000,156.0,Fredrikinkatu 16 A,Punavuori,Helsinki,"Fredrikinkatu 16 A, Helsinki",9551.0
3,1123200,108.0,Punavuorenkatu 26/ Telakkakatu 16 C 51,Punavuori,Helsinki,"Punavuorenkatu 26/ Telakkakatu 16 C 51, Helsinki",10400.0
4,2941500,159.0,Hietalahdenranta 2 A 69,Punavuori,Helsinki,"Hietalahdenranta 2 A 69, Helsinki",18500.0


In [37]:
# I make a new df called 'smaller_than_800', where I rule out apartments bigger than 800 m2.
smaller_than_800 = df7[df7['size'] < 800].sort_values('size', ascending=False)
smaller_than_800

# Make new column of street
# Creating a new column 'street' by extracting characters before any digits in 'street_address'
# ^ starts from the beginning of value, [^\d] commands the code to find anything exept a number ( d=digit)
smaller_than_800['street'] = smaller_than_800['street_address'].str.extract(r'^([^\d]+)')

# Remove any trailing spaces from the extracted street names
smaller_than_800['street'] = smaller_than_800['street'].str.strip()

# Street_and_city
smaller_than_800['steet_and_city'] = smaller_than_800['street'] + ', ' + smaller_than_800['city']

smaller_than_800.head()

# The number of rows shrinks from previous.

,price,size,street_address,borough,city,address,e/m²,street,steet_and_city
371,8800000,726.0,Marjaniemenranta,Marjaniemi,Helsinki-Uusimaa,"Marjaniemenranta, Helsinki-Uusimaa",12121.0,Marjaniemenranta,"Marjaniemenranta, Helsinki-Uusimaa"
340,2950000,700.0,Angelniementie 185,Angelniemi,Salo,"Angelniementie 185, Salo",4214.0,Angelniementie,"Angelniementie, Salo"
385,1290000,600.0,Sattulantie 68,Sattula,Hattula,"Sattulantie 68, Hattula",2150.0,Sattulantie,"Sattulantie, Hattula"
496,1750000,582.0,Rusthollinrinne 4,Tasanne,Tampere,"Rusthollinrinne 4, Tampere",3007.0,Rusthollinrinne,"Rusthollinrinne, Tampere"
79,4900000,545.0,Kuusiniementie 5,Kuusisaari,Helsinki,"Kuusiniementie 5, Helsinki",8991.0,Kuusiniementie,"Kuusiniementie, Helsinki"


In [39]:
# Making a new df to check, how many times each street is recurring in the data

# Create a new df with 'street' and 'city' columns
street_and_city = smaller_than_800[['street', 'city']].copy()

common_streets = street_and_city.groupby(['street', 'city']).size().reset_index(name='count').sort_values(by='count', ascending=False)

# Combine 'street' and 'city' into a new column, called 'address'.
common_streets['address'] = common_streets['street'] + ', ' + common_streets['city']

print(common_streets[common_streets['count'] >= 5])
print("- - - - - - - -")
print(f"There are {len(common_streets[common_streets['count'] >= 5])} streets, on which there are at least 5 expensive flats for sale.")

# Saving results into csv.
common_streets.to_csv("common_streets_smaller_than_800_240804.csv")

                 street      city  count                       address
106  Konttinosturinkuja  Helsinki     13  Konttinosturinkuja, Helsinki
249         Tehtaankatu  Helsinki     11         Tehtaankatu, Helsinki
51           Horisontti  Helsinki     11          Horisontti, Helsinki
157            Merikatu  Helsinki     10            Merikatu, Helsinki
47     Hietalahdenranta  Helsinki     10    Hietalahdenranta, Helsinki
288        Westendintie     Espoo     10           Westendintie, Espoo
68    Jousenpuistonkatu     Espoo      9      Jousenpuistonkatu, Espoo
65        Itätuulenkuja     Espoo      8          Itätuulenkuja, Espoo
260         Tuuliniitty     Espoo      7            Tuuliniitty, Espoo
34        Fredrikinkatu  Helsinki      7       Fredrikinkatu, Helsinki
262          Töllinmäki  Helsinki      6          Töllinmäki, Helsinki
265         Unioninkatu  Helsinki      6         Unioninkatu, Helsinki
21        Capellanranta  Helsinki      5       Capellanranta, Helsinki
259   

In [40]:
# Then doing a new df about flats smaller than 800 square meters by cities.
# Using smaller_than_800 dataframe.

# In which city there are most listing?
# To get both columns after value_counts into a csv, I need to turn results into a df with reset_index().
listings_per_city = smaller_than_800.city.value_counts().reset_index()
# Naming the columns.
listings_per_city.columns = ['city', 'count']

# Just checking the number is now smaller than earlier when checking the cities.
print(len(listings_per_city))

# Save result to its own csv.
listings_per_city.to_csv('listings-per-city-smaller-than-800_240804.csv', index=False)

42


I want to do a scatterplot to my story, so I make a df for that purpose.

In [41]:
# For a scatter plot, make df of all listings size below 800 m2 with 'price', 'size' and 'city'.
# I take the city just in case, if I want to highlight Helsinki based listings.
scatterplot = smaller_than_800[['price', 'size', 'city']].copy()

# Change 'price' into simpler format by dividing with 1,000,000.
scatterplot['price'] = scatterplot['price']/1000000

print(scatterplot)

scatterplot.to_csv("price-and-size-to-scatterplot_240804.csv", index=False)


     price   size              city
371  8.800  726.0  Helsinki-Uusimaa
340  2.950  700.0              Salo
385  1.290  600.0           Hattula
496  1.750  582.0           Tampere
79   4.900  545.0          Helsinki
..     ...    ...               ...
342  1.469   78.0          Helsinki
138  1.275   75.5          Helsinki
91   1.090   75.0          Helsinki
271  2.200   74.0          Helsinki
444  1.190   48.5            Vantaa

[490 rows x 3 columns]
